In [1]:
#keras_timeseries.py
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from math import sqrt
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten    
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
#from keras.layers.convolutional import Convolution2D
#from keras.layers.convolutional import MaxPooling2D
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras import optimizers
#from keras import backend as K

df = pd.read_csv("customers_timeseries_updated.csv")
#df = pd.read_csv("data-1000customers.csv")
#df = pd.read_csv("100_timeseries_checking_to_share.csv")
#print (df.columns.values)
#df = df.iloc[:,0:3]
#use column names
df = df[['client_debtor_number','dates','fv_cost']]
#print("new dataset columns ",df.columns.values)
#df.head()
# number of customers
len(df['client_debtor_number'].unique())

Using TensorFlow backend.
/Users/anusha/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


101

In [2]:
df_1015130 = df[df['client_debtor_number'] == 1015193]
del df_1015130['client_debtor_number']


In [3]:
#df[df.Date.str.match(r'^2018-06')]
#df[df[\"col\"].str.contains('this|that')==False]
# splitting train and test datasets 
#training 2010-2017, testing on 2018 data\
train_x = df_1015130[df_1015130['dates'].str.contains('/2018') == False]
test_x = df_1015130[df_1015130['dates'].str.contains('/2018') == True]
#print(test_x.dates)
train_x_cust = train_x['fv_cost']
test_x_cust = test_x['fv_cost']

print("train_data ",train_x_cust)
print("test_data ", test_x_cust)

train_data  0           0.000000
1           0.000000
2           0.000000
3           0.000000
4           0.000000
5           0.000000
6           0.000000
7           0.000000
8           0.000000
9           0.000000
10          0.000000
11          0.000000
12          0.000000
13          0.000000
14          0.000000
15          0.000000
16          0.000000
17          0.000000
18          0.000000
19          0.000000
20          0.000000
21          0.000000
22          0.000000
23          0.000000
24          0.000000
25          0.000000
26          0.000000
27          0.000000
28          0.000000
29          0.000000
             ...    
307581    190.301116
307582    193.302938
307583    196.305162
307584    197.021261
307585    199.928856
307586    191.000154
307587    194.007212
307588    194.524591
307589    197.375700
307590    200.080716
307591     61.073114
307592     62.782689
307593     64.492494
307594     66.202527
307595     67.912788
307596     67.921500
3

In [4]:

def lag_feature(df, lag=1):
    if not type(df) == pd.DataFrame:
        df = pd.DataFrame(df, columns=['fv_cost'])
    
    def rename_lag(ser, j):
        ser.name = ser.name + f'_{j}'
        return ser
        
    # add a column lagged by `i` steps
    for i in range(1, lag + 1):
        df = df.join(df.fv_cost.shift(i).pipe(rename_lag, i))

    df.dropna(inplace=True)
    return df

def prepare_training_data(series_data, lag):
    " Converts a series of data into a lagged, scaled sample."
    # scale training data
    scaler = MinMaxScaler(feature_range=(-1, 1))
    cost_vals = scaler.fit_transform(series_data.values.reshape(-1, 1))
    
    # convert series to lagged features
    cost_lagged = lag_feature(cost_vals, lag=lag)

    # X, y format taking the first column (original time series) to be the y
    X = cost_lagged.drop('fv_cost', axis=1).values
    y = cost_lagged.fv_cost.values
    
    # keras expects 3 dimensional X
    X = X.reshape(X.shape[0], 1, X.shape[1])
    
    return X, y, scaler

train_x_cust,train_y_cust, scalar_train = prepare_training_data(train_x['fv_cost'], 3)
test_x_cust,test_y_cust,scalar_test = prepare_training_data(test_x['fv_cost'], 3)
print(train_x_cust.shape)
print(train_y_cust.shape)


(3189, 1, 3)
(3189,)


In [5]:

lag =  3
# model parameters
num_neurons = 50 #24
batch_size = 1  # this forces the lstm to step through each time-step one at a time
batch_input_shape=(batch_size, 1, lag)
dropout_rate =0.2
# instantiate a sequential model
model = Sequential()
model.add(Conv1D(filters=num_neurons,batch_size=1, kernel_size=3, strides=3, 
                 padding="same",activation='linear',dilation_rate=1, 
                 input_shape=(1, 3),data_format='channels_first'))
model.add(MaxPooling1D(pool_size=3,strides=3, padding="same"))
model.add(Dropout(dropout_rate))
# add LSTM layer - stateful MUST be true here in 
# order to learn the patterns within a series
model.add(LSTM(units=num_neurons, 
              batch_input_shape=batch_input_shape, return_sequences=False,# as we only want last hidden output 
              stateful=True))
model.add(Dense(1,activation='linear'))
from keras import optimizers
adam = optimizers.Adam(lr=0.01, decay=0.01)
model.compile(loss='mean_squared_error', optimizer=adam)

In [6]:
model.fit(train_x_cust, train_y_cust, epochs=10, batch_size=batch_size, verbose=2, shuffle=False)

Epoch 1/10
 - 49s - loss: 0.0323
Epoch 2/10
 - 47s - loss: 0.0189
Epoch 3/10
 - 47s - loss: 0.0185
Epoch 4/10
 - 47s - loss: 0.0175
Epoch 5/10
 - 47s - loss: 0.0170
Epoch 6/10
 - 47s - loss: 0.0174
Epoch 7/10
 - 47s - loss: 0.0174
Epoch 8/10
 - 48s - loss: 0.0170
Epoch 9/10
 - 48s - loss: 0.0174
Epoch 10/10
 - 48s - loss: 0.0171


In [7]:

# Estimate model performance
# default batch size is 32 so we need to give batch size explicitly if we want different batch size or 
# input size should be divisible by batch size for stateful LSTM
trainScore = model.evaluate(train_x_cust, train_y_cust, batch_size=1, verbose=2)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
testScore = model.evaluate(test_x_cust, test_y_cust, batch_size=1, verbose=2)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

print('unscaled test score %f MSE' %(scalar_test.inverse_transform(testScore)))
yhat_act_test = scalar_test.inverse_transform(testScore)
yhat_act_train = scalar_train.inverse_transform(trainScore)
#print (math.sqrt(scaler.inverse_transform(testScore)))
#print ("actual train mean_squared_error ",mean_squared_error(test_x.fv_cost[-(len(yhat_act_test)):], yhat_act_test))
print ("actual test rmse ",sqrt(mean_squared_error(test_x.fv_cost[-(len(yhat_act_test)):], yhat_act_test)))
print ("actual train rmse ",sqrt(mean_squared_error(train_x.fv_cost[-(len(yhat_act_train)):],yhat_act_train)))
#scaler.inverse_transform(testScore)

Train Score: 0.01 MSE (0.12 RMSE)
Test Score: 0.09 MSE (0.30 RMSE)
unscaled test score 724.851458 MSE
actual test rmse  56.8891378239897
actual train rmse  46.3822232086311


In [8]:
# generate predictions for training
trainPredict = model.predict(train_x_cust,batch_size=1)
testPredict = model.predict(test_x_cust,batch_size=1)

# shift train predictions for plotting
trainPredictPlot = np.empty_like(df_1015130)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[lag:len(trainPredict)+lag, :] = trainPredict
 
# shift test predictions for plotting
testPredictPlot = np.empty_like(df_1015130)
testPredictPlot[:, :] = np.nan
#testPredictPlot[len(trainPredict)+(lag*2)+1:len(df_1015130)-1, :] = testPredict


In [11]:
# inverse scaling for a forecasted value
cust_id = "100019"
#y_inv = invert_scale(scaler, test_x_cust, testPredict)
yhat_act = scalar_test.inverse_transform(testPredict)
rmse = sqrt(mean_squared_error(test_x.fv_cost[-(len(yhat_act)):], yhat_act))
print(cust_id," Test RMSE: ",rmse)

100019  Test RMSE:  212.9444811085056


In [12]:
# write predictions into csv
result_prediction = pd.DataFrame()#, original_fv : test_y_cust, prediction_fv :testPredict})
#result_prediction['dates'] = test_x['dates']
result_prediction['original_fv'] = test_y_cust
result_prediction['prediction_fv'] = testPredict

result_prediction.to_csv("predictions_for_client_1015193.csv")

In [40]:
# predictions for timesteps ahead
# 7,14 ,21 timesteps ahead
#we need atleast 4 elements to prepare training data
futureElement = testPredict[-4:]

def prepare_training_data_array(series_data, lag):
    " Converts a series of data into a lagged, scaled sample."
    # scale training data
    scaler = MinMaxScaler(feature_range=(-1, 1))
    cost_vals = scaler.fit_transform(series_data.reshape(-1, 1))
    
    # convert series to lagged features
    cost_lagged = lag_feature(cost_vals, lag=lag)

    # X, y format taking the first column (original time series) to be the y
    X = cost_lagged.drop('fv_cost', axis=1).values
    y = cost_lagged.fv_cost.values
    
    # keras expects 3 dimensional X
    X = X.reshape(X.shape[0], 1, X.shape[1])
    
    return X, y, scaler

futureElement_x,futureElement_y,scalar = prepare_training_data_array(futureElement, 3)
print(futureElement_x.shape)
x=futureElement[-4:]
#def timestep_ahead(num,model,futureElement):
#futureElement = futureElement_x
def timestep_ahead(num,model,futureElement,x):
    futureElements = []
    unscaled_prediction =[]
    futureElements.append(futureElement.tolist())
    for i in range(num):
        futureElement_x,futureElement_y,scalar = prepare_training_data_array(futureElement[-4:], 3)
        print(futureElement_x.shape,"shape after preparing data in 1st iteration")
        print("future element value after preparing data",futureElement_x)
        futureElement = model.predict(futureElement_x,batch_size=1,verbose=2)
        unscaled_prediction.append(scalar.inverse_transform(futureElement).tolist())
        print("unscaled prediction ",unscaled_prediction)
        futureElements.append(futureElement)
        print("future element before appending x",futureElement)
        futureElement = np.append(x,futureElement)
        x=futureElement[-4:]
        print("future element after appending x",futureElement)
        print("futureElements ",futureEltestPredict[-4:])
        print("iteration ",i,"  complete")     
    return (futureElement[-1],futureElements,unscaled_prediction)



(1, 1, 3)


In [ ]:
_7time_ahead,elements,unscaled_prediction = timestep_ahead(7,model,futureElement,x)
print ("scaled prediction value on 7th day ahead ",_7time_ahead)
_14time_ahead,elements,unscaled_prediction  = timestep_ahead(14,model,futureElement,x)
print ("scaled prediction value on 14th day ahead ",_14time_ahead)
_21time_ahead,elements,unscaled_prediction  = timestep_ahead(21,model,futureElement,x)
print ("scaled prediction value on 21st day ahead ",_21time_ahead)

In [ ]:
# 3months ahead predicitons
_91days_ahead,elements,unscaled_prediction = timestep_ahead(91,model,futureElement,x)
print ("scaled prediction value on 91st day ahead ",_91days_ahead)


In [50]:
print ("fe_x ",futureElement_x)
print ("fe_y ",futureElement_y)
print ("fe ",futureElement)
print ("fe_x shape ",futureElement_x.shape)
print ("fe shape ",futureElement.shape)
testPredict[-4:] #Do not scale when Future Element is TestPredict

fe_x  [[[ 0.15081406 -1.          1.        ]]]
fe_y  [0.8552742]
fe  [[0.2667769 ]
 [0.23281354]
 [0.2523563 ]
 [0.2643192 ]]
fe_x shape  (1, 1, 3)
fe shape  (4, 1)


array([[0.2667769 ],
       [0.23281354],
       [0.2523563 ],
       [0.2643192 ]], dtype=float32)

In [62]:
X=testPredict[-4:]
print (X)
print (X.shape)# we want 4 values if we need y as well as 3 are sufficient
X=X.reshape(X.shape[1], 1, X.shape[0])
print ("x ",X)
print ("x shape ",X.shape)

[[0.2667769 ]
 [0.23281354]
 [0.2523563 ]
 [0.2643192 ]]
(4, 1)
x  [[[0.2667769  0.23281354 0.2523563  0.2643192 ]]]
x shape  (1, 1, 4)


Experimentation on Future values of train dataset

In [66]:
x=futureElement[-3:]
print (x)
print ("shape ",x.shape)
x=x.reshape(-1, 1)
print ("shape -1 ",x.shape)
x.reshape(x.shape[1], 1, x.shape[0])

307608    375.067774
307609    378.479600
307610    381.891882
Name: fv_cost, dtype: float64
shape  (3,)
shape -1  (3, 1)


/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.


array([[[375.067774 , 378.4796001, 381.8918823]]])

In [69]:
# predictions for timesteps ahead
# 7,14 ,21 timesteps ahead
#we need atleast 4 elements to prepare training data
futureElement = testPredict[-4:]
# if using testpredict as futureElement no need to scale data again using MinMaxscalar
futureElement = train_x.fv_cost[-4:]
 
#futureElement_x,futureElement_y,scalar = prepare_training_data_array(futureElement, 3)
#print(futureElement_x.shape)
#x=futureElement[-4:]
#def timestep_ahead(num,model,futureElement):
#futureElement = futureElement_x
def timestep_ahead(num,model,futureElement,x):
    #Since we are using train we need to scale the input
    futureElement_x,futureElement_y,scalar = prepare_training_data_array(futureElement[-4:], 3)
    print ("shape of fe_x before for loop ",futureElement_x.shape)
    futureElements = []
    unscaled_prediction =[]
    futureElements.append(futureElement.tolist())
    # plus 1 because one value for futureElement_y 
    for i in range(num+1):
        print(futureElement_x.shape,"shape after preparing data in ",i," iteration")
        print("future element value after preparing data",futureElement_x)
        futureElement = model.predict(futureElement_x,batch_size=1,verbose=2)
        unscaled_prediction.append(scalar.inverse_transform(futureElement).tolist())
        print("unscaled prediction ",unscaled_prediction)
        futureElements.append(futureElement.tolist())
        print("future element before appending x",futureElement)
        futureElement = np.append(x,futureElement)
        x=futureElement[-3:]
        futureElement_x=x.reshape(-1, 1)
        futureElement_x=futureElement_x.reshape(
            futureElement_x.shape[1], 1, futureElement_x.shape[0])
        print("future element after appending x",futureElement)
        print("futureElements ",futureElements)     
        print("iteration ",i,"  complete")     
    return (futureElement[-1],futureElements,unscaled_prediction)


In [70]:
#comparing future elements of train data with test dataset to check model performance
_7ahead,elements,unscaled_prediction = timestep_ahead(7,model,train_x.fv_cost[-4:],x)
print ("prediction of 7days ahead ",_7ahead," unscaled prediction ",unscaled_prediction[-1]," actual value ",test_x.fv_cost.iloc[7])

shape of fe_x before for loop  (1, 1, 3)
(1, 1, 3) shape after preparing data in  0  iteration
future element value after preparing data [[[ 0.33324421 -0.33342245 -1.        ]]]
unscaled prediction  [[[376.7713317871094]]]
future element before appending x [[-0.00054966]]
future element after appending x [ 3.75067774e+02  3.78479600e+02  3.81891882e+02 -5.49659133e-04]
futureElements  [[371.656404, 375.06777400000004, 378.4796001, 381.8918823], [[-0.0005496591329574585]]]
iteration  0   complete
(1, 1, 3) shape after preparing data in  1  iteration
future element value after preparing data [[[ 3.78479600e+02  3.81891882e+02 -5.49659133e-04]]]
unscaled prediction  [[[376.7713317871094]], [[379.7027587890625]]]
future element before appending x [[0.5722511]]
future element after appending x [ 3.78479600e+02  3.81891882e+02 -5.49659133e-04  5.72251081e-01]
futureElements  [[371.656404, 375.06777400000004, 378.4796001, 381.8918823], [[-0.0005496591329574585]], [[0.5722510814666748]]]
iter

/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # Remove the CWD from sys.path while we load stuff.


In [72]:
_14ahead,elements,unscaled_prediction = timestep_ahead(14,model,train_x.fv_cost[-4:],x)
print ("prediction of 14days ahead ",_14ahead," unscaled prediction ",unscaled_prediction[-1]," actual value ",test_x.fv_cost.iloc[14])


shape of fe_x before for loop  (1, 1, 3)
(1, 1, 3) shape after preparing data in  0  iteration
future element value after preparing data [[[ 0.33324421 -0.33342245 -1.        ]]]
unscaled prediction  [[[376.7951354980469]]]
future element before appending x [[0.00410614]]
future element after appending x [3.75067774e+02 3.78479600e+02 3.81891882e+02 4.10614163e-03]
futureElements  [[371.656404, 375.06777400000004, 378.4796001, 381.8918823], [[0.004106141626834869]]]
iteration  0   complete
(1, 1, 3) shape after preparing data in  1  iteration
future element value after preparing data [[[3.78479600e+02 3.81891882e+02 4.10614163e-03]]]
unscaled prediction  [[[376.7951354980469]], [[379.7044372558594]]]
future element before appending x [[0.5725769]]
future element after appending x [3.78479600e+02 3.81891882e+02 4.10614163e-03 5.72576880e-01]
futureElements  [[371.656404, 375.06777400000004, 378.4796001, 381.8918823], [[0.004106141626834869]], [[0.5725768804550171]]]
iteration  1   compl

/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # Remove the CWD from sys.path while we load stuff.


In [76]:
_21ahead,elements,unscaled_prediction = timestep_ahead(42,model,train_x.fv_cost[-4:],x)
print ("prediction of 21days ahead ",_21ahead," unscaled prediction ",
       unscaled_prediction[-1]," actual value ",test_x.fv_cost.iloc[42])

/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # Remove the CWD from sys.path while we load stuff.


shape of fe_x before for loop  (1, 1, 3)
(1, 1, 3) shape after preparing data in  0  iteration
future element value after preparing data [[[ 0.33324421 -0.33342245 -1.        ]]]
unscaled prediction  [[[376.6831359863281]]]
future element before appending x [[-0.01778766]]
future element after appending x [ 3.75067774e+02  3.78479600e+02  3.81891882e+02 -1.77876577e-02]
futureElements  [[371.656404, 375.06777400000004, 378.4796001, 381.8918823], [[-0.01778765767812729]]]
iteration  0   complete
(1, 1, 3) shape after preparing data in  1  iteration
future element value after preparing data [[[ 3.78479600e+02  3.81891882e+02 -1.77876577e-02]]]
unscaled prediction  [[[376.6831359863281]], [[379.7035217285156]]]
future element before appending x [[0.5723951]]
future element after appending x [ 3.78479600e+02  3.81891882e+02 -1.77876577e-02  5.72395086e-01]
futureElements  [[371.656404, 375.06777400000004, 378.4796001, 381.8918823], [[-0.01778765767812729]], [[0.5723950862884521]]]
iteratio

unscaled prediction  [[[376.6831359863281]], [[379.7035217285156]], [[377.89495849609375]], [[376.26611328125]], [[379.31866455078125]], [[377.3958435058594]], [[376.5306091308594]], [[378.9433898925781]], [[376.9833068847656]], [[376.845703125]], [[378.6791076660156]], [[376.6285095214844]], [[377.171142578125]], [[378.4486083984375]], [[376.340576171875]], [[377.4889831542969]], [[378.2375183105469]], [[376.1291198730469]], [[377.772216796875]], [[378.00616455078125]], [[376.00250244140625]], [[378.005126953125]], [[377.734375]], [[375.9726867675781]], [[378.1542053222656]], [[377.436279296875]], [[376.0406799316406]], [[378.228759765625]], [[377.1144104003906]], [[376.2076416015625]], [[378.2386474609375]], [[376.78094482421875]], [[376.4776306152344]], [[378.1941833496094]], [[376.47259521484375]], [[376.8399963378906]], [[378.1158142089844]], [[376.207275390625]], [[377.2073059082031]], [[377.9987487792969]], [[376.00634765625]], [[377.5557861328125]]]
future element before append

In [36]:
daily_pred =[np.array(arr).tolist() for arr in elements]
daily_pred
def flatten(l): return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]
pred_list = flatten(daily_pred)

import csv 
myFile = open('daily_predictions_3months.csv', 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerow(pred_list)

pred_list

[371.656404,
 375.06777400000004,
 378.4796001,
 381.8918823,
 -0.06496140360832214,
 0.7212505340576172,
 0.6659982204437256,
 -0.34120362997055054,
 -0.6369178295135498,
 0.4477390944957733,
 0.6127439737319946,
 0.5911823511123657,
 -0.5209013819694519,
 -0.66240394115448,
 -0.686203122138977,
 0.21877828240394592,
 0.40478774905204773,
 0.6499253511428833,
 -0.6026910543441772,
 -0.5530111789703369,
 -0.6654351949691772,
 -0.6978175640106201,
 -0.24336612224578857,
 0.2983408272266388,
 0.6923211812973022,
 0.6095691919326782,
 -0.6121375560760498,
 -0.6669450998306274,
 -0.6886459589004517,
 -0.7030854225158691,
 -0.6590583324432373,
 0.2039482593536377]

In [37]:
#find every 7th element into a list
week_end_pred = pred_list[0::7]
week_end_pred

[371.656404,
 -0.34120362997055054,
 -0.686203122138977,
 -0.6978175640106201,
 -0.6886459589004517]

In [38]:
# weekly average for 3 months
weekly_avg_pred = [(sum(pred_list[x:x+7]))/7 for x in range(0, len(pred_list), 7)]
weekly_avg_pred

[215.48827825012762,
 -0.07282305189541408,
 -0.17626416683197021,
 -0.08857644881520953,
 -0.26383449350084576]

# find average for every 7 days
import itertools
n=7 #7days
list_week = list(itertools.chain.from_iterable([i]*7 for i in [sum(pred_list[i:i+7])//7 for i in range(0,len(pred_list),7)]))
list_week


#for i in range(7):
futureElement = model.predict(futureElement,batch_size=1)
futureElements.append(futureElement)
futureElement = np.append(futureElement, futureElements[0][0][0][:3])
x=futureElement[:3]
print(x)
print("iteration 0 complete")
#    #return futureElements
print(futureElement)
print(futureElements)

futureElement_x,futureElement_y,scalar = prepare_training_data_array(futureElement, 3)
print(futureElement_x.shape,"shape after preparing data in 1st iteration")
print("future element value after preparing data",futureElement_x)
futureElement = model.predict(futureElement_x,batch_size=1)
futureElements.append(futureElement)
print("future element before appending x",futureElement)
futureElement = np.append(futureElement,x)
x=futureElement[:3]
print("future element after appending x",futureElement)
print("iteration 1 complete")
print(futureElements)

futureElement_x,futureElement_y,scalar = prepare_training_data_array(futureElement, 3)
print(futureElement_x.shape,"shape after preparing data in 1st iteration")
print("future element value after preparing data",futureElement_x)
futureElement = model.predict(futureElement_x,batch_size=1)
futureElements.append(futureElement)
print("future element before appending x",futureElement)
futureElement = np.append(futureElement,x)
x=futureElement[:3]
print("future element after appending x",futureElement)
print("iteration 2 complete")
print(futureElements)

print(type(futureElements))
print(type(futureElement))

In [ ]:
flat_list = [x for sublist in futureElements for x in sublist]
print(flat_list[0][0][1])

#above function loops in below format
#for sublist in futureElements:
#    for x in sublist:
#        flat_list.append(x)

In [ ]:
#price.resample('1W')  
#aapl.resample('W').mean()
#http://benalexkeen.com/resampling-time-series-data-with-pandas/
#resampling based on days and then slicing every 7th day
#ts.resample('D').interpolate()[::7]

In [ ]:
#df.last('3M') #last 3 months data
# to sort dates column before getting last 3months
#df_sorted = df.sort_values(by="Date",ascending=True) \
#    .set_index("Date")
#    .last("3M")

https://stats.stackexchange.com/questions/305863/how-to-train-lstm-model-on-multiple-time-series-data

https://stats.stackexchange.com/questions/67362/shall-i-use-weekly-or-monthly-data-for-forecast

In [ ]:
#weekly predicitons
#weekly data
week_data = df_1015130.set_index('dates').resample('1W').mean()
#week_data['dates'] = pd.to_datetime(week_data['dates'], errors='coerce')


week_data = week_data.resample('1W')


In [ ]:
#week_data['dates'] = pd.to_datetime(week_data['dates'], unit='D',utc=True)
week_data = df_1015130
##week_data['New']=week_data.dates.map(week_data.set_index('dates').iloc[1:].resample('D').sum().rolling(7,min_periods =1).visit.mean()).shift()
week_data['dates'] = pd.to_datetime(week_data['dates'])


In [ ]:
week_data.set_index('dates',inplace=True)
week_data_resample = week_data.resample('1W').mean()

In [ ]:
week_data_resample.describe()

In [ ]:
#c_index = week_data_resample.index("/2018")
#l2 = l[:c_index]
week_data_resample['dates'] = week_data_resample.index
week_data_resample['dates']=week_data_resample['dates'].astype(str)
week_data_resample = week_data_resample.fillna(0)

In [ ]:
train_week = week_data_resample[week_data_resample['dates'].str.contains('2018') == False]
test_week = week_data_resample[week_data_resample['dates'].str.contains('2018') == True]

train_x_week,train_y_week, scalar_train = prepare_training_data(train_week['fv_cost'], 3)
test_x_week,test_y_week,scalar_test = prepare_training_data(test_week['fv_cost'], 3)
print(train_x_week.shape)
print(train_y_week.shape)


In [ ]:
train_week

In [ ]:
week_data_resample

In [ ]:
lag =  3
num_neurons = 24
batch_size = 1  # this forces the lstm to step through each time-step one at a time
batch_input_shape=(batch_size, 1, lag)

# instantiate a sequential model
model = Sequential()
model.add(Conv1D(filters=24,batch_size=1, kernel_size=3, strides=3, padding="same",activation='relu',dilation_rate=1, input_shape=(1, 3),data_format='channels_first'))
model.add(MaxPooling1D(pool_size=3,strides=3, padding="same"))
model.add(Dropout(0.2))
model.add(LSTM(units=num_neurons, 
              batch_input_shape=batch_input_shape, return_sequences=False,# as we only want last hidden output 
              stateful=True))
model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='adam')

In [ ]:
model.fit(train_x_week, train_y_week, epochs=10, batch_size=batch_size, verbose=2, shuffle=False)

In [ ]:
trainScore = model.evaluate(train_x_week, train_y_week, batch_size=1, verbose=2)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
testScore = model.evaluate(test_x_week, test_y_week, batch_size=1, verbose=2)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))
print('unscaled test score %f MSE' %(scalar_test.inverse_transform(testScore)))
scalar_train.inverse_transform(trainScore)


In [ ]:
# generate predictions for training
trainPredict = model.predict(train_x_week,batch_size=1)
testPredict = model.predict(test_x_week,batch_size=1)

In [ ]:
futureElement = testPredict[-4:]

futureElement_x,futureElement_y,scalar = prepare_training_data_array(futureElement, 3)
print(futureElement_x.shape)
x=futureElement[:3]
#def timestep_ahead(num,model,futureElement):
#futureElement = futureElement_x
def timestep_ahead_week(num,model,futureElement,x):
    futureElements = []
    futureElements.append(futureElement.tolist())

    for i in range(num):
        futureElement_x,futureElement_y,scalar = prepare_training_data_array(futureElement, 3)
        print(futureElement_x.shape,"shape after preparing data in 1st iteration")
        print("future element value after preparing data",futureElement_x)
        futureElement = model.predict(futureElement_x,batch_size=1,verbose=2)
        futureElements.append(futureElement)
        print("future element before appending x",futureElement)
        futureElement = np.append(futureElement,x)
        x=futureElement[:3]
        print("future element after appending x",futureElement)
        print("iteration ",i,"  complete")
    return (futureElement[0],futureElements)



In [ ]:
_12_weeks_ahead,all_pred = timestep_ahead_week(12,model,futureElement,x)
print ("scaled prediction value on 3rd month ahead ",_12_weeks_ahead)
print ("all predicitons ",all_pred)

In [ ]:
# converting list of arrays to lists
#l = [arr.tolist() for arr in l]
#[l.tolist() for l in list1]

#weekly_pred = [arr.tolist() for arr in all_pred] #list object has no attribute tolist
weekly_pred =[np.array(arr).tolist() for arr in all_pred]
weekly_pred

In [ ]:
def flatten(l): return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]

In [ ]:
pred_list = flatten(weekly_pred)
pred_list

In [ ]:
csvwriter.writerow(pred_list)

In [ ]:
import csv 
myFile = open('weekly_predictions_3months.csv', 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerow(pred_list)

experiemnt code for outlier behaving customer model

In [ ]:
# predictions and analysis for customer with highest rmse 1681085
df1 = df[df.fv_cost.notnull()]
cust_data = df1[df1['client_debtor_number'] == 1681085]
del cust_data['client_debtor_number']


lag =  3

# model parameters
num_neurons = 24
batch_size = 1  # this forces the lstm to step through each time-step one at a time
batch_input_shape=(batch_size, 1, lag)

# instantiate a sequential model
model = Sequential()
model.add(Conv1D(filters=24,batch_size=1, kernel_size=3, strides=3, padding="same",activation='relu',dilation_rate=1, input_shape=(1, 3),data_format='channels_first'))
model.add(MaxPooling1D(pool_size=3,strides=3, padding="same"))
model.add(Dropout(0.2))
model.add(LSTM(units=num_neurons, 
              batch_input_shape=batch_input_shape, return_sequences=False,# as we only want last hidden output 
              stateful=True))
model.add(Dense(1))
# we can add dropoutlayer after dense as well again
model.add(Dropout(0.2))
# compile
model.compile(loss='mean_absolute_error', optimizer='adam')



train_x = cust_data[cust_data['dates'].str.contains('/2018') == False]
test_x = cust_data[cust_data['dates'].str.contains('/2018') == True]
print(train_x.head(), len(train_x))
print (test_x.head(),len(test_x))

train_x_cust,train_y_cust, scalar_train = prepare_training_data(train_x['fv_cost'], 3)
test_x_cust,test_y_cust,scalar_test = prepare_training_data(test_x['fv_cost'], 3)  
model.fit(train_x_cust, train_y_cust, epochs=10, batch_size=batch_size, verbose=2, shuffle=False)
    # input size should be divisible by batch size for stateful LSTM
trainScore = model.evaluate(train_x_cust, train_y_cust, batch_size=1, verbose=2)
testScore = model.evaluate(test_x_cust, test_y_cust, batch_size=1, verbose=2)
train_rmse = trainScore ###
test_rmse = testScore ###
    # generate predictions for training
trainPredict = model.predict(train_x_cust,batch_size=1)
testPredict = model.predict(test_x_cust,batch_size=1)
yhat_act_test = scalar_test.inverse_transform(testPredict)
    # report performance
rmse_test = sqrt(mean_squared_error(test_x.fv_cost[-(len(yhat_act_test)):], yhat_act_test))
print(cust_id," Test RMSE: ",rmse_test)
actual_error_scores_test = rmse_test ####
yhat_act_train = scalar_train.inverse_transform(trainScore)
rmse_train = sqrt(mean_squared_error(train_x.fv_cost[-(len(yhat_act_train)):],yhat_act_train))
actual_train_rmse = rmse_train #####
    # write predictions into csv
result_prediction = pd.DataFrame()#, original_fv : test_y_cust, prediction_fv :testPredict})
    #result_prediction['dates'] = test_x['dates']
result_prediction['original_fv_scaled'] = test_y_cust
result_prediction['prediction_scaled'] = testPredict
result_prediction['prediction_actual'] = yhat_act_test
result_prediction['fv_actual'] = test_x.fv_cost[-(len(yhat_act_test)):]
result_prediction.to_csv("predictions_1/predictions_for_client_1681085.csv")